In [1]:
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, util
import re

model = SentenceTransformer('all-mpnet-base-v2')


/home/marcuswrrn/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
def get_json_data(filename : str):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

gpt_prompts = get_json_data("./Data/jobs_prompts.json")
gpt_responses = get_json_data("./Data/gpt_job_responses.json")

print(f"Num Prompts: {len(gpt_prompts)}\nNum Responses: {len(gpt_responses)}")

Num Prompts: 13447
Num Responses: 13447


In [3]:
def get_response_msgs(model_data):
    return [response["choices"][0]["message"]["content"] for response in model_data]

def get_tokens(model_data):
    tokens = [response["usage"] for response in model_data]
    prompts = []
    resp = []
    total = []
    for val in tokens:
        prompts.append(val["prompt_tokens"])
        resp.append(val["completion_tokens"])
        total.append(val["total_tokens"])
    return prompts, resp, total

response_msgs = get_response_msgs(gpt_responses)
prompt_tokens, resp_tokens, total_tokens = get_tokens(gpt_responses)


In [4]:
def split_positive_negative(input_list):
    positive_negative_list = []
    for input_str in input_list:
        # Find the start of positive and negative sections
        positive_start = input_str.find("Positive:")
        negative_start = input_str.find("Negative:")

        # Extract the positive and negative sentences
        positive = input_str[positive_start + len("Positive:"):negative_start].strip()
        negative = input_str[negative_start + len("Negative:"):].strip()

        # Remove leading characters such as '-' or bullet points
        if positive.startswith('-'):
            positive = positive[1:].strip()
        if negative.startswith('-'):
            negative = negative[1:].strip()

        positive_negative_list.append((positive, negative))

    return positive_negative_list

# Build dataframe
df = pd.read_csv("./Data/Job_Titles.csv")
# Get Neutral sentences
df["Neutral"] = [prompt["content"] for prompt in gpt_prompts]
# Get Positive and negative sentences
vals = split_positive_negative(response_msgs)
df["Positive"] = [resp[0] for resp in vals]
df["Negative"] = [resp[1] for resp in vals]
# Get token information
df["Prompt_Tokens"] = prompt_tokens
df["Response_Tokens"] = resp_tokens
df["Total_Tokens"] = total_tokens
df.head(10)

,Job_Titles,Neutral,Positive,Negative,Prompt_Tokens,Response_Tokens,Total_Tokens
0,Technical Support Representative,I am a Technical Support Representative,I am knowledgeable and skilled in providing te...,I may encounter difficult and frustrated custo...,62,29,91
1,A&H Underwriter,I am an A&H Underwriter,I have a great job as an A&H Underwriter.,I am not happy with my job as an A&H Underwriter.,65,35,100
2,A&H Underwriting Manager,I am an A&H Underwriting Manager,I have a high-level position as an A&H Underwr...,"I am not just an entry-level employee, but I h...",65,44,109
3,A&P Technician,I am an A&P Technician,Being an A&P Technician is a rewarding career ...,The job of an A&P Technician can be physically...,63,55,118
4,ABA Therapist,I am an ABA Therapist,I provide effective therapy to children with a...,I do not enjoy working as an ABA Therapist and...,63,37,100
5,ABAP Developer,I am an ABAP Developer,I have expertise in ABAP development and can c...,I do not have experience in other programming ...,62,44,106
6,AEM Architect,I am an AEM Architect,I have extensive knowledge and experience in A...,I do not have experience in other content mana...,63,30,93
7,AEM Cloud Site Reliability Engineer,I am an AEM Cloud Site Reliability Engineer,Being an AEM Cloud Site Reliability Engineer i...,The job of an AEM Cloud Site Reliability Engin...,67,72,139
8,AEM Developer,I am an AEM Developer,I have expertise in developing websites using ...,I do not have experience in developing website...,63,35,98
9,AEM Front End Developer,I am an AEM Front End Developer,I have expertise in AEM Front End Development.,I do not have experience in any other Front En...,65,30,95


In [5]:
def generate_encodings_for_all_sents(df):
    # Create encodings
    print("Encoding Neutral")
    df["Neutral_Encodings"] = generate_encodings(df, "Neutral")
    print("Encoding Positive")
    df["Positive_Encodings"] = generate_encodings(df, "Positive")
    print("Encoding Negative")
    df["Negative_Encodings"] = generate_encodings(df, "Negative")


def generate_encodings(df, Column):
    return df[Column].apply(lambda x: model.encode(x))


#df["Neutral_Terminated"] = df["Neutral"].apply(lambda x: f"{x}.")
df["Positive_Leading"] = df["Positive"].apply(lambda x: x.replace(".", ""))
df["Negative_Leading"] = df["Negative"].apply(lambda x: x.replace(".", ""))
print(df["Positive_Leading"][5])
df["Positive_Leading_Encodings"] = generate_encodings(df,"Positive_Leading")
print("Starting Negative")
df["Negative_Leading_Encodings"] = generate_encodings(df,"Negative_Leading")
print("Finished")

I have expertise in ABAP development and can create efficient programs
Starting Negative
Finished


In [6]:
df.head()

,Job_Titles,Neutral,Positive,Negative,Prompt_Tokens,Response_Tokens,Total_Tokens,Positive_Leading,Negative_Leading,Positive_Leading_Encodings,Negative_Leading_Encodings
0,Technical Support Representative,I am a Technical Support Representative,I am knowledgeable and skilled in providing te...,I may encounter difficult and frustrated custo...,62,29,91,I am knowledgeable and skilled in providing te...,I may encounter difficult and frustrated custo...,"[0.040035043, -0.05122371, -0.036133047, -0.02...","[0.06889098, 0.053461976, -0.03753226, 0.01474..."
1,A&H Underwriter,I am an A&H Underwriter,I have a great job as an A&H Underwriter.,I am not happy with my job as an A&H Underwriter.,65,35,100,I have a great job as an A&H Underwriter,I am not happy with my job as an A&H Underwriter,"[-0.030297037, 0.09025022, -0.012940765, -0.07...","[-0.026900822, 0.07068688, -0.010721647, -0.07..."
2,A&H Underwriting Manager,I am an A&H Underwriting Manager,I have a high-level position as an A&H Underwr...,"I am not just an entry-level employee, but I h...",65,44,109,I have a high-level position as an A&H Underwr...,"I am not just an entry-level employee, but I h...","[-0.014753003, 0.06767135, 0.0007165152, -0.07...","[-0.019844484, 0.054371767, -0.0030334315, -0...."
3,A&P Technician,I am an A&P Technician,Being an A&P Technician is a rewarding career ...,The job of an A&P Technician can be physically...,63,55,118,Being an A&P Technician is a rewarding career ...,The job of an A&P Technician can be physically...,"[-0.028020246, 0.03969528, -0.038768895, -0.02...","[-0.03767958, -0.030884342, -0.049201913, -0.0..."
4,ABA Therapist,I am an ABA Therapist,I provide effective therapy to children with a...,I do not enjoy working as an ABA Therapist and...,63,37,100,I provide effective therapy to children with a...,I do not enjoy working as an ABA Therapist and...,"[0.015004761, 0.009011287, -0.05012279, 0.0174...","[0.0030206721, -0.02781823, -0.05529693, -0.02..."


In [7]:
def append_to_df(df, columns, file_read):
    df_main = pd.read_json(file_read)
    for column in columns:
        df_main[column] = df[column]
    return df_main
        

df_main = append_to_df(df, ["Positive_Leading", "Positive_Leading_Encodings", "Negative_Leading", "Negative_Leading_Encodings"], "/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Data_Processing/Data/job_processed_data_main.json")

#df_main.to_json("./Data/job_processed_data_main.json")
df_main.head()

,Job_Titles,Neutral,Positive,Negative,Prompt_Tokens,Response_Tokens,Total_Tokens,Neutral_Encodings,Positive_Encodings,Negative_Encodings,Neutral_Terminated,Neutral_Terminated_Encodings,Positive_Leading,Positive_Leading_Encodings,Negative_Leading,Negative_Leading_Encodings
0,Technical Support Representative,I am a Technical Support Representative,I am knowledgeable and skilled in providing te...,I may encounter difficult and frustrated custo...,62,29,91,"[0.052540335800000004, -0.0741394237, -0.02414...","[0.0398985222, -0.039076753000000006, -0.03879...","[0.0750338361, 0.055929131800000004, -0.034461...",I am a Technical Support Representative.,"[0.0467863977, -0.0582639724, -0.0304021016000...",I am knowledgeable and skilled in providing te...,"[0.040035043, -0.05122371, -0.036133047, -0.02...",I may encounter difficult and frustrated custo...,"[0.06889098, 0.053461976, -0.03753226, 0.01474..."
1,A&H Underwriter,I am an A&H Underwriter,I have a great job as an A&H Underwriter.,I am not happy with my job as an A&H Underwriter.,65,35,100,"[-0.0183508322, 0.025549562600000002, -0.02162...","[-0.0315244012, 0.0867658779, -0.0072295303, -...","[-0.027420181800000002, 0.0644791871, -0.00949...",I am an A&H Underwriter .,"[-0.022203857100000002, 0.0287630353, -0.02035...",I have a great job as an A&H Underwriter,"[-0.030297037, 0.09025022, -0.012940765, -0.07...",I am not happy with my job as an A&H Underwriter,"[-0.026900822, 0.07068688, -0.010721647, -0.07..."
2,A&H Underwriting Manager,I am an A&H Underwriting Manager,I have a high-level position as an A&H Underwr...,"I am not just an entry-level employee, but I h...",65,44,109,"[-0.017012547700000002, 0.0080151027, -0.02787...","[-0.0027675340000000003, 0.068974562, -0.00189...","[-0.0194694344, 0.0587330759, -0.0063610324, -...",I am an A&H Underwriting Manager.,"[-0.0162954777, 0.0157897249, -0.0263584778000...",I have a high-level position as an A&H Underwr...,"[-0.014753003, 0.06767135, 0.0007165152, -0.07...","I am not just an entry-level employee, but I h...","[-0.019844484, 0.054371767, -0.0030334315, -0...."
3,A&P Technician,I am an A&P Technician,Being an A&P Technician is a rewarding career ...,The job of an A&P Technician can be physically...,63,55,118,"[-0.040869303, -0.0479970723, -0.0461201258, -...","[-0.027997929600000002, 0.0399987362, -0.03595...","[-0.0378840007, -0.030123831700000003, -0.0423...",I am an A&P Technician .,"[-0.0417694934, -0.0297952704, -0.0478198938, ...",Being an A&P Technician is a rewarding career ...,"[-0.028020246, 0.03969528, -0.038768895, -0.02...",The job of an A&P Technician can be physically...,"[-0.03767958, -0.030884342, -0.049201913, -0.0..."
4,ABA Therapist,I am an ABA Therapist,I provide effective therapy to children with a...,I do not enjoy working as an ABA Therapist and...,63,37,100,"[0.0211608782, -0.0087794699, -0.0511314794, -...","[0.014801985600000001, 0.0252737999, -0.049951...","[0.0058508902, -0.0159540195, -0.0475744903, -...",I am an ABA Therapist.,"[0.015887849000000002, 0.0088945329, -0.048975...",I provide effective therapy to children with a...,"[0.015004761, 0.009011287, -0.05012279, 0.0174...",I do not enjoy working as an ABA Therapist and...,"[0.0030206721, -0.02781823, -0.05529693, -0.02..."


In [8]:
df_main.to_json("./Data/job_processed_data_main.json")